In [1]:
#%matplotlib notebook
import uproot as ur
import matplotlib.pyplot as plt
import k3d
import numpy as np
import awkward as ak


## Get file and TTree, print branches, convert to array

In [2]:
piplus_events = ur.open("rec_pionplus.root:events")
pi0_events = ur.open("rec_pion0.root:events")

print(piplus_events.keys())

['mcparticles2', 'mcparticles2/mcparticles2.ID', 'mcparticles2/mcparticles2.g4Parent', 'mcparticles2/mcparticles2.reason', 'mcparticles2/mcparticles2.mask', 'mcparticles2/mcparticles2.steps', 'mcparticles2/mcparticles2.secondaries', 'mcparticles2/mcparticles2.pdgID', 'mcparticles2/mcparticles2.status', 'mcparticles2/mcparticles2.colorFlow[2]', 'mcparticles2/mcparticles2.genStatus', 'mcparticles2/mcparticles2.charge', 'mcparticles2/mcparticles2.spare[1]', 'mcparticles2/mcparticles2.spin[3]', 'mcparticles2/mcparticles2.vsx', 'mcparticles2/mcparticles2.vsy', 'mcparticles2/mcparticles2.vsz', 'mcparticles2/mcparticles2.vex', 'mcparticles2/mcparticles2.vey', 'mcparticles2/mcparticles2.vez', 'mcparticles2/mcparticles2.psx', 'mcparticles2/mcparticles2.psy', 'mcparticles2/mcparticles2.psz', 'mcparticles2/mcparticles2.pex', 'mcparticles2/mcparticles2.pey', 'mcparticles2/mcparticles2.pez', 'mcparticles2/mcparticles2.mass', 'mcparticles2/mcparticles2.time', 'mcparticles2/mcparticles2.properTime', 

In [3]:
def get_vector(events,varname='HcalEndcapHits',energy='energy'):
    E = events["%s.%s"%(varname,energy)].array(library='np')
    x = events["%s.position.x"%varname].array(library='np')
    y = events["%s.position.y"%varname].array(library='np')
    z = events["%s.position.z"%varname].array(library='np')
    #theta = events["%s.polar.theta"%varname].array(library='np')
    #phi = events["%s.polar.phi"%varname].array(library='np')

    return E,x, y, z, theta, phi

def get_truth(gen_events):
    pdgID = gen_events['mcparticles2.pdgID'].array(library='np')
    status = gen_events['mcparticles2.status'].array(library='np')
    genStatus = gen_events['mcparticles2.genStatus'].array(library='np')
    pex = gen_events['mcparticles2.psx'].array(library='np')
    pey = gen_events['mcparticles2.psy'].array(library='np')
    pez = gen_events['mcparticles2.psz'].array(library='np')
    mass = gen_events['mcparticles2.mass'].array(library='np')

    return pdgID,status, genStatus,pex,pey,pez,mass

In [4]:
E = {}
x = {}
y = {}
z  = {}
theta={}
phi = {}


## Get data

# get hits

In [5]:

for i in ['HcalHadronEndcap','EcalEndcapP']:#,'TrackerEndcap','VertexEndcap']:
    tag = 'piplus_' + i
    E[tag], x[tag], y[tag],z[tag],theta[tag],phi[tag] = get_vector(piplus_events,varname="%sHitsReco"%i)
    tag = 'pi0_' + i
    E[tag], x[tag], y[tag],z[tag],theta[tag],phi[tag] = get_vector(pi0_events,varname="%sHitsReco"%i)



## get MC

In [6]:
def get_GeneratedValues(events):
    mc = {} 
    mc['pdgID'], mc['status'], mc['genStatus'], mc['pex'],mc['pey'], mc['pez'], mc['mass']= get_truth(events)
    for i in range(len(mc['pdgID'][0])):
        if(mc['genStatus'][0][i] !=1): continue
        px = mc['pex'][0][i]
        py =  mc['pey'][0][i]
        pz = mc['pez'][0][i]
        pt =  np.sqrt(px*px + py*py)
        p = np.sqrt(px*px+py*py+pz*pz)
        theta = np.arccos(pz/p)*180/np.pi
        print('MC particle ID ', mc['pdgID'][0][i],  'genStatus ', mc['genStatus'][0][i],
               ' px = %2.2f, py=%2.2f, pz=%2.2f'%(mc['pex'][0][i],mc['pey'][0][i],mc['pez'][0][i]))
        print('p = %2.2f , theta =%2.2f'%(p, theta))
    return p, theta

truth = {}
truth['piplus_p'], truth['piplus_theta'] = get_GeneratedValues(piplus_events)
truth['pi0_p'], truth['pi0_theta'] = get_GeneratedValues(pi0_events)



MC particle ID  211 genStatus  1  px = -1.41, py=-3.99, pz=9.07
p = 10.00 , theta =25.00
MC particle ID  111 genStatus  1  px = -3.64, py=-2.15, pz=9.07
p = 10.01 , theta =25.00


## Get clusters

In [7]:
#for i in ['HcalBarrelClusters','HcalHadronEndcapClusters','EcalBarrelClusters','EcalEndcapClusters']:
#    E[i], x[i], y[i],z[i] = get_vector("%s"%i,energy='energy')

In [8]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.colors import LogNorm

In [9]:
print(x.keys())

dict_keys(['piplus_HcalHadronEndcap', 'pi0_HcalHadronEndcap', 'piplus_EcalEndcapP', 'pi0_EcalEndcapP'])


## Plot 2D transverse view for hits in endcap

In [10]:
#loop over events


data = []

for ievt in range(3,4):
    for part_type in ['piplus']:#pi0
         #print(ievt)
        ECAL = []
        print (part_type)

        key = '%s_EcalEndcapP'%(part_type)
        
        
#        mask = [(E[key][ievt]>0.005) & (E[key][ievt]/np.sum(E[key][ievt])>0.001)]
        mask = [(E[key][ievt]>0.01)]
   
        ECAL_x = x[key][ievt][mask]
        ECAL_y = y[key][ievt][mask]
        ECAL_E = E[key][ievt][mask]*1000.0 #(MeV)
        
        print('NUmber of CELLS ', len(ECAL_E))
    
        #print(cell_x)
        #print(cell_E)
        #c = ax[0].hist2d(cell_x,cell_y, weights=cell_E,label=part_type + ' evt #%i'%(ievt), 
        #             bins=[200,200],range=([[-2000, 2000], [-2000,2000]]))
        #fig.colorbar(c[3], ax=ax[0])            
        suma_ECAL = np.sum(E[key][ievt])
        print(part_type , ' Suma ECAL =%2.2f GeV'%suma_ECAL)
        #### HCAL 
        
        #ECAL.append(list(cell_E))
        
        key = '%s_HcalHadronEndcap'%(part_type)

        print('######################   Entering HCAL   ##########################')
        
        event_array = []
        for length_1 in range(100,500,20):
            for length_2 in range(100,500,20):
                Image = {}
                Image['ECAL_E'] = ECAL_E
                Image['ECAL_x'] = ECAL_x
                Image['ECAL_y'] = ECAL_y
                Image['HCAL1_E'] = []
                Image['HCAL2_E'] = []
                Image['HCAL3_E'] = []
                Image['HCAL1_x'] = []
                Image['HCAL2_x'] = []
                Image['HCAL3_x'] = []
                Image['HCAL1_y'] = []
                Image['HCAL2_y'] = []
                Image['HCAL3_y'] = []
                Image['boundary'] = (length_1, length_2)
                if(length_1+length_2>999): continue
                #fig,ax  = plt.subplots(1,4,figsize=(20,4),sharex=True, sharey=True)
    
                print('Length 1= ', length_1,' legnth 2' , length_2)
              
                for iz, zbins in enumerate([(3000,3000+length_1),(3000+length_1, 3000+length_1+length_2),(3000+length_1+length_2,4000)]):
                    print('############')
                    mask = [(z[key][ievt]>zbins[0]) & (z[key][ievt]<zbins[1]) & (E[key][ievt]>0.0)]
                    if(np.sum(mask)<1): continue
                    cell_x = x[key][ievt][mask]
                    cell_y = y[key][ievt][mask]
                    cell_E = E[key][ievt][mask] *1000.0 # MEV
                    print(cell_E)
                    #c = ax[iz+1].hist2d(cell_x,cell_y, weights=cell_E,label=part_type + ' evt #%i'%(ievt), 
                    #    bins=[40,40],range=([[-2000, 2000], [-2000,2000]]))
                    #fig.colorbar(c[3], ax=ax[iz+1])       
                    Image['HCAL%i_E'%(iz+1)].append(cell_E)
                    Image['HCAL%i_x'%(iz+1)].append(cell_x)
                    Image['HCAL%i_y'%(iz+1)].append(cell_y)

                    #ax[iz+1].set_title('%i, %i'%(zbins[0],zbins[1]))
                    #HCAL['%i_x'%(iz+1)].append(list(cell_x))
                    #HCAL['%i_y'%(iz+1)].append(list(cell_y))

        #HCAL_tot = [HCAL['1'],HCAL['2'],HCAL['3'],(3200,3400)]  
                #print([HCAL,(length_1,length_2)])
                event_array.append(Image)
        
        data.append(event_array)
        ##END OF EVENT
        #ax[0].set_title('ECAL \n %s event'%(part_type))
        #for iz in range(3):
        #    ax[iz+1].set_title('HCAL %i \n %s event, %2.0f GeV'%(iz,part_type,truth[part_type+'_p']))

        #ax[0].legend()
        #for a in ax:
        #    a.set_xticklabels([])
        #    a.set_yticklabels([])
        #plt.subplots_adjust(wspace=0, hspace=0.5)
#TOTAL = []
with open('test.npy', 'wb') as f:
    np.save(f, data)
    
#print(data[ievt])
#print(data[ievt])

#print('Primera entry')
#print(data[0][1])
#print('Second entry')

#print(data[0][2])
#print('Thriy')

#print(data[0][3])

#print ('HCAL 1', HCAL['1'])
#print ('HCAL 2 ', HCAL['2'])
#print ('HCAL 3 ', HCAL['3'])

piplus
NUmber of CELLS  29
piplus  Suma ECAL =1.52 GeV
######################   Entering HCAL   ##########################
Length 1=  100  legnth 2 100
############
[ 2.5650024   0.56762695  0.18005371  0.72631836  0.3173828   0.390625
  0.22888184  0.72021484  0.12054443  0.8666992   0.29296875  0.16021729
  0.9536743   0.46539307  2.770996    3.4484863   0.47302246  0.44403076
  0.12207031  0.08850098  0.11444092  0.2822876  16.105652    0.07629395
  0.2609253   0.45776367  1.0925293   0.18005371  0.1083374   0.09460449
  0.0793457   0.10528564  0.3112793   0.07781982  0.9277344   0.579834
  2.7526855 ]
############
[ 0.1663208   0.19378662  0.25024414  0.95977783  0.8682251  12.667847
  2.256775    0.20141602  1.1383057   0.19989014  0.11749268  0.44250488
  0.18157959  0.21362305  0.54626465  2.3040771   1.0620117   1.8875122
  9.268188    0.28381348  0.1876831 ]
############
[0.26397705 0.37078857 2.7404785  0.3173828  0.15563965 1.7593384
 0.27313232 4.5043945  0.6072998 ]
Length

[ 2.5650024   0.56762695  0.18005371  0.72631836  0.1663208   0.19378662
  0.25024414  0.3173828   0.390625    0.22888184  0.72021484  0.95977783
  0.12054443  0.8666992   0.8682251  12.667847    0.29296875  0.16021729
  0.9536743   0.46539307  2.256775    2.770996    0.20141602  3.4484863
  0.47302246  0.44403076  0.12207031  0.08850098  0.11444092  1.1383057
  0.2822876   0.19989014  0.11749268 16.105652    0.07629395  0.2609253
  0.45776367  1.0925293   0.44250488  0.18005371  0.1083374   0.09460449
  0.18157959  0.0793457   0.10528564  0.21362305  0.3112793   0.07781982
  0.9277344   0.579834    2.7526855   0.54626465  2.3040771   1.0620117
  4.5043945   1.8875122   9.268188    0.28381348  0.1876831 ]
############
[0.26397705 0.37078857 2.7404785  0.3173828  0.15563965 1.7593384
 0.27313232 0.6072998 ]
############
Length 1=  240  legnth 2 300
############
[ 2.5650024   0.56762695  0.18005371  0.72631836  0.1663208   0.19378662
  0.25024414  0.3173828   0.390625    0.22888184  0.72

  0.28381348  0.1876831 ]
############
[0.26397705 0.37078857 2.7404785  1.7593384  0.27313232]
############
Length 1=  320  legnth 2 460
############
[ 2.5650024   0.56762695  0.18005371  0.72631836  0.1663208   0.19378662
  0.25024414  0.3173828   0.390625    0.22888184  0.72021484  0.95977783
  0.12054443  0.8666992   0.8682251  12.667847    0.29296875  0.16021729
  0.9536743   0.46539307  2.256775    2.770996    0.20141602  3.4484863
  0.47302246  0.44403076  0.12207031  0.08850098  0.11444092  1.1383057
  0.2822876   0.19989014  0.11749268 16.105652    0.07629395  0.2609253
  0.45776367  1.0925293   0.44250488  0.18005371  0.1083374   0.09460449
  0.18157959  0.0793457   0.10528564  0.21362305  0.3112793   0.07781982
  0.3173828   0.15563965  0.9277344   0.579834    2.7526855   0.54626465
  2.3040771   1.0620117   4.5043945   1.8875122   0.6072998   9.268188
  0.28381348  0.1876831 ]
############
[0.26397705 0.37078857 2.7404785  1.7593384  0.27313232]
############
Length 1=  320 

<ipython-input-10-3fcaac4cc208>:18: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  ECAL_x = x[key][ievt][mask]
<ipython-input-10-3fcaac4cc208>:19: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  ECAL_y = y[key][ievt][mask]
<ipython-input-10-3fcaac4cc208>:20: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  ECAL_E = E[key


############
[ 2.5650024   0.56762695  0.18005371  0.72631836  0.1663208   0.19378662
  0.25024414  0.3173828   0.390625    0.22888184  0.72021484  0.95977783
  0.12054443  0.8666992   0.8682251  12.667847    0.29296875  0.16021729
  0.9536743   0.46539307  2.256775    2.770996    0.20141602  3.4484863
  0.47302246  0.44403076  0.12207031  0.08850098  0.11444092  1.1383057
  0.2822876   0.19989014  0.11749268 16.105652    0.07629395  0.2609253
  0.45776367  1.0925293   0.44250488  0.18005371  0.1083374   0.09460449
  0.18157959  0.0793457   0.10528564  0.21362305  0.3112793   0.07781982
  0.37078857  0.3173828   0.15563965  0.9277344   0.579834    2.7526855
  0.54626465  2.3040771   1.0620117   1.7593384   4.5043945   1.8875122
  0.6072998   9.268188    0.28381348  0.1876831 ]
############
[0.26397705 2.7404785  0.27313232]
############
Length 1=  360  legnth 2 100
############
[ 2.5650024   0.56762695  0.18005371  0.72631836  0.1663208   0.19378662
  0.25024414  0.3173828   0.390625 

In [ ]:
with open('test.npy', 'rb') as f:
    data = np.load(f,allow_pickle=True)
    ievt = 0
    print('number of images in event', len(data[ievt]))
    ## Here we loop over all "images", which are created by integrating HCAL sampling layers defing 3 sectors defined 
    ## by two z position that define boundary. Note for all images ECAL is the same (no longitudinal separation in ECAL)
    for im in range(len(data[ievt])):
        print('#  IMAGE NUMBER %i############\n\n\n'%im)
        #print(data[ievt][im]) #This is a dictionary
        print(data[ievt][im].keys())
        print('HCAL1 x [mm] \n', data[ievt][im]['HCAL1_x']) ##This is x of HCAL#1 image im
        print('HCAL1 y [mm]\n', data[ievt][im]['HCAL1_y']) ##This is y of HCAL#1 image im
        print('HCAL1 E [GeV]\n', data[ievt][im]['HCAL1_E']) ## this is E of HCAL#1 image im
        print('HCAL2 x [mm]\n', data[ievt][im]['HCAL2_x']) ##This is x of HCAL#2 image im
        print('HCAL2 y [mm]\n', data[ievt][im]['HCAL2_y']) ##This is y of HCAL#2 image im
        print('HCAL2 E [mm]\n', data[ievt][im]['HCAL2_E']) ## this is E of HCAL#2 image im
        print('HCAL3 x [mm]\n',data[ievt][im]['HCAL3_x']) ##This is x of HCAL#3 image im
        print('HCAL3 y [mm]\n',data[ievt][im]['HCAL3_y']) ##This is y of HCAL#3 image im
        print('HCAL3 E [mm]\n',data[ievt][im]['HCAL3_E']) ## this is E of HCAL#3 image im
        print('ECAL x [mm]\n',data[ievt][im]['ECAL_E']) ## this is E of ECAL image im
        print('ECAL y [mm]\n',data[ievt][im]['ECAL_x']) ## this is x of ECAL image im
        print('ECAL E [mm]\n',data[ievt][im]['ECAL_y']) ## this is x of ECAL image im
        print('Boundaries\n',data[ievt][im]['boundary'])  ## This are the boundaries (z positions of long layers) used to created image


number of images in event 400
#  IMAGE NUMBER 0############



dict_keys(['ECAL_E', 'ECAL_x', 'ECAL_y', 'HCAL1_E', 'HCAL2_E', 'HCAL3_E', 'HCAL1_x', 'HCAL2_x', 'HCAL3_x', 'HCAL1_y', 'HCAL2_y', 'HCAL3_y', 'boundary'])
HCAL1 x [mm] 
 [array([ -652.12441103,  -555.5318284 ,  -774.59889817,  -800.48080268,
        -678.00631554,  -600.36060201,  -503.76801938,  -529.64992389,
        -722.83508915,  -626.24250652,  -826.36270719,  -729.77012456,
        -555.5318284 ,  -678.00631554,  -362.34666314,  -767.66386276,
        -529.64992389,  -555.5318284 ,  -897.07338531,  -916.02025441,
        -871.1914808 ,  -826.36270719,  -388.22856765,  -395.16360307,
        -369.28169856,  -298.57102044,  -395.16360307,  -465.87428118,
       -1045.42977696,  -948.83719433,  -614.23067283,  -491.75618569,
        -614.23067283,  -755.65202907,  -845.30957629,  -845.30957629,
        -845.30957629])]
HCAL1 y [mm]
 [array([1274.65044327, 1300.53234778, 1345.36112139, 1441.95370402,
       1371.2430259 , 

In [ ]:
#sf_ECAL = 0.253
#sf_HCAL = 0.025
    #print('Energy = %2.3f'%(suma_ECAL/sf_ECAL+suma_HCAL/sf_HCAL))
    ##suma_HCAL = np.sum(E[key][ievt])
    #print(suma_HCAL/sf_HCAL)